# Deploy Fraud detector using code

This notebook shows you how to use the sample template code base and deploy Amazon Frauddetector using mock training data. If you have your own training data, then you can substitute the mock data with your data.

## Setup



In [ ]:
import os

# Optional, if you want to change the default region or profile
# os.environ["AWS_DEFAULT_REGION"]='us-west-2'
# os.environ["AWS_PROFILE"]='default'

In [ ]:
# -- AWS stuff -- 
import boto3

# Standard python
import os
import os, sys
sys.path.append('./src')
import datetime


account_id =  boto3.client('sts').get_caller_identity().get('Account')


### ACTION: [SPECIFY BUCKET]

In [ ]:
#Specify your bucket here and bucket prefix
fd_data_bucket ="databucket-mlopsdemo-202013"
fd_data_bucket_prefix = "fraudetector-demo"

Configure fraud detector data access role

Configure this role for fraud detector to have read access to your data. In this example policy, Amazon Fraud detector has read access to objects in bucket whose names contain *frauddetector*

```json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "BucketLevelAccess",
      "Effect": "Allow",
      "Action": [
        "s3:ListBucket",
        "s3:GetBucketLocation"
      ],
      "Resource": "arn:aws:s3:::*frauddetector*"
    },
    {
      "Sid": "ObjectAccess",
      "Effect": "Allow",
      "Action": "s3:GetObject",
      "Resource": "arn:aws:s3:::*frauddetector*/*"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "frauddetector.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```

### ACTION: [SPECIFY ROLE]

In [ ]:
# Specify the role..
frauddetector_role_name = "DemoStackUSWest2-S1EnvironmentBootstrap-FraudDetectorRole"

#### General config

In [ ]:

s3_uri_data = "s3://{}/{}".format(fd_data_bucket, fd_data_bucket_prefix)

In [ ]:

frauddetector_role_arn="arn:aws:iam::{}:role/{}".format(account_id, frauddetector_role_name)

In [ ]:
import logging
logging.basicConfig(level="INFO", handlers=[logging.StreamHandler(sys.stdout)],
                        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [ ]:
local_sample_file=os.path.join("src", "sample_data_variable_creation.csv")

## Data Transformation

This sample creates mock data to get started..

In [ ]:
%%time

from main_demo_data_transformer import DemoDataTransformer

transformer = DemoDataTransformer()
transformer.run_pipeline(s3_destination_uri=s3_uri_data)


In [ ]:
import pandas as pd


df = pd.read_csv(s3_uri_data + "/train.csv")
df.sample(n=10)

#### Mini sample data to dynamically create variables
Create a mini copy of the sample data so that the Fraud Detector variables can be inferred from the data in the next step

In [ ]:
# Save a copy small subset of data in src , so it can be used to create the Fraud detector variables
transformer.write_csv(df.sample(n=10, random_state=42), local_sample_file)

## Create model -  Train with your dataset 

In [ ]:

model_name = "frauddetector_demo_mlops"
model_version = None
wait = True
model_description="This is a demo model"
event_type_name = "demoevent-mlops"
entity="democustomer-mlops"

In [ ]:
model_type = "ONLINE_FRAUD_INSIGHTS"

In [ ]:
%%time

from core.fraud_detector_train import FraudDetectorTrain
from features.feature_variables_dynamic import FeatureVariablesDynamic
from core.fraud_detector_event import FraudDetectorEvent
import pandas as pd

# Step 1: Define features
model_variables = FeatureVariablesDynamic(pd.read_csv(local_sample_file), true_labels=[1])

# Step 2: Define event
model_event = FraudDetectorEvent()
model_event.create_event(event_type_name=event_type_name, description="This is a demo event", entity=entity,
                             model_variables=model_variables)

# Step 3: Run training
trainer = FraudDetectorTrain()
model_details = trainer.run(model_name=model_name,model_type=model_type,event_type_name=event_type_name,
                            role_arn = frauddetector_role_arn, model_variables=model_variables, 
                            model_description=model_description, 
                            s3_training_file=s3_uri_data, wait=True)


In [ ]:

def get_model(model_name, model_version):
    client = boto3.client('frauddetector')
    
    response = client.describe_model_versions(
    modelId=model_name,
    modelVersionNumber=model_version,
    modelType='ONLINE_FRAUD_INSIGHTS')
    
    return response


In [ ]:
model_version = model_details["modelVersionNumber"]
get_model(model_name, model_version)


## Create Detector  - To predictions on new data

In [ ]:
model_to_deploy_name, model_to_deploy_version = model_name , model_version or "1.0"

In [ ]:
detector_name = "demo_detector"

In [ ]:
from rules.detector_rule_model_score_positive import DetectorRuleModelScorePositive

detector_rules = [
        DetectorRuleModelScorePositive(rule_id="positivescorerule", model_name=model_to_deploy_name, threshold=2.6)
]
    

In [ ]:
%%time

from core.fraud_detector_model_based_deploy import FraudDetectorModelBasedDeploy

deployer = FraudDetectorModelBasedDeploy()
models_details = [
    {"modelId": model_to_deploy_name,
     "modelDescription": model_description,
     "modelType": model_type,
     "modelVersionNumber": model_to_deploy_version}
]
detector = deployer.deploy(detector_name=detector_name, 
                           detector_rules=detector_rules, 
                           event_type_name=event_type_name,
                           detector_description="This is a demo detector",
                           model_versions=models_details)



In [ ]:
detector

### Predict

In [ ]:
detector_version = detector["detectorVersionId"]


In [ ]:
def predict(detector_id, detector_version, event_id, attributes_dict, event_timestamp, event_type_name, entity, entity_id):
    client = boto3.client('frauddetector')
    response = client.get_event_prediction(
    detectorId= detector_id ,
    eventTypeName = event_type_name,
    detectorVersionId=detector_version,
    eventId = event_id,
    entities = [{'entityType':entity, 'entityId':entity_id}],
    eventTimestamp=event_timestamp,
    eventVariables=attributes_dict
)
    return response

In [ ]:
from datetime import timezone, datetime

predict(detector_name, detector_version=detector_version, 
        event_id="1", 
        event_timestamp= datetime.utcnow().isoformat(timespec='seconds') + "Z",
        event_type_name=event_type_name, 
        entity= entity,
        entity_id = "demo1",
        attributes_dict= {"ip": "122.233.333.333", 
                          "email":"edum@a.com", 
                          }
       )

## CloudWatch metrics

This shows you the list of models used in the last 7 days

In [ ]:
import json


def get_models_used(used_in_days):
    import boto3
    from datetime import datetime, timedelta


    # Create CloudWatch client
    cloudwatch = boto3.client('cloudwatch')

    models_used_in_n_days = []

    # List ModelInvocation metric for AWS/FraudDetector
    paginator = cloudwatch.get_paginator('list_metrics')
    for response in paginator.paginate(MetricName='GetEventPrediction',
                                       Namespace='AWS/FraudDetector'):

        # For metric available query stats
        for metric in response["Metrics"]:
            metric_dim = metric["Dimensions"]

            resp_metric_stats = cloudwatch.get_metric_statistics(Namespace='AWS/FraudDetector',
                                                                 MetricName='GetEventPrediction',
                                                                 Dimensions=metric_dim,
                                                                 StartTime=datetime.now() - timedelta(days=used_in_days),
                                                                 EndTime=datetime.now(),
                                                                 Period=60 * 60 * 24 * used_in_days,
                                                                 Statistics=[
                                                                     'SampleCount'
                                                                 ]
                                                                 )

            models_used_in_n_days.append({"metric_dim": metric_dim, "data": resp_metric_stats["Datapoints"]})

    return models_used_in_n_days

In [ ]:
models_used_in_last_n_days = get_models_used(used_in_days = 7)

print(json.dumps(models_used_in_last_n_days, indent=1, default=str))

## [Optional] Delete demo detector + undeploy model
This is so that you do not incur unexpected charges due to demo models running in your account

In [ ]:
from core.fraud_detector_undeploy import FraudDetectorUndeploy

deploy = FraudDetectorUndeploy()
deploy.delete_detector(detector_name)


In [ ]:
deploy.undeploy_model(model_to_deploy_name, model_to_deploy_version)